In [1]:
import boto3
import pykx as kx

<frozen importlib._bootstrap>:228: PyKXWarning: Failed to initialize PyKX fully licensed functionality.
To access all functionality of PyKX please download an evaluation license from https://kx.com/kdb-insights-personal-edition-license-download/
Full installation instructions can be found at https://code.kx.com/pykx/getting-started/installing.html
Falling back to unlicensed mode, which has limited functionality.
Refer to https://code.kx.com/pykx/user-guide/advanced/modes.html for more information on licensed vs unlicensed modalities.
 Captured output from initialization attempt:
    '2023.07.18T12:01:07.768 licence error: exp


In [2]:
# enter your environment details and user here
environmentId = "rtci3jl7pii6tyhp5d2shk"
userName = "finTorq_user"
clusterName="finTorq_hdb"
region="eu-west-1"

In [3]:
def getConnectionString(userName, clusterName):
    defaultSession = boto3.Session()
    finspace = defaultSession.client('finspace')

    userDetails = finspace.get_kx_user(userName=userName, environmentId=environmentId)
    iamRole = userDetails['iamRole']
    userArn = userDetails['userArn']

    sts = defaultSession.client('sts')
    creds = sts.assume_role(RoleArn=iamRole, RoleSessionName="connect_kdb_session")
    creds = creds['Credentials']

    connectSession = boto3.Session(aws_access_key_id=creds['AccessKeyId'],
                                 aws_secret_access_key=creds['SecretAccessKey'],
                                 aws_session_token=creds['SessionToken']
                                 )

    connect = connectSession.client('finspace')
    connString = connect.get_kx_connection_string(environmentId=environmentId,
                                                  userArn=userArn,
                                                  clusterName=clusterName
                                                  )

    return connString.get('signedConnectionString', None)

In [4]:
getConnectionString(userName, clusterName)

':tcps://vpce-02b4d0f70aab47141-idnyk9ae.vpce-svc-07988fe84c80b9cc6.eu-west-1.vpce.amazonaws.com:443:finTorq_user:Host=vpce-02b4d0f70aab47141-idnyk9ae.vpce-svc-07988fe84c80b9cc6.eu-west-1.vpce.amazonaws.com&Port=5000&User=finTorq_user&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEOz%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCWV1LXdlc3QtMSJIMEYCIQC6OmljAN0me6Fig%2BumUtti1txC%2FCpARcdqLhd82Q1D%2BQIhAKBVE8Iosz7P58Muoc0ddUDZ0%2Fp4ShtDzKHSC3D%2FCpSNKvgCCHUQAhoMNzY2MDEyMjg2MDAzIgwdC53N3RVOpvye77kq1QJiADSjqmjqfcoLTou%2FeTqu9IlB1GNTqQax2eMUzqGMDdVMD9jTGTEjmzsS8ijnwpqfBCN%2BAboJqMcKJOEaE9Bppe48hFBAnZbbZerGhMXgucs6xSgjxCaO2INyG%2BfI9w6Y2B3c8KX0wijHjxyLOU9HRijA12e%2FgscFjydmxtqbh2vtrjmuWxNMuV9CpZA0awFkVHmDkmaGtA1aCOASFG684nieB5qJ97PXTRs8vXYu6LwGRNU7KqncrSk0BDSF7seOjXTreLrgvBDvmYpq0tIB29izZ0eOGYkhUzi1op15FKX7cUBj1zkeU%2FdVLrkU2YmHg8nMmSb5IImWKaG6ccYpHY%2FNYPQDqnPhrjsO0eQoT%2Frg0rJIWZrFWZ2p%2FjaqaDRwYVcNUIBOrMpH0FYSCPA%2B4DXduDXIWBnRMPG3hhOVTqQP4WztWDa3i72VgwzfYMDO5BIvSTCJ%2FtmlBjq

In [5]:
def getPyKxConn(userName, clusterName):
    connString = getConnectionString(userName, clusterName)
    connParts = connString.split(":")

    host=connParts[2].strip("/")
    port = int(connParts[3])
    username=connParts[4]
    password=connParts[5]
    print("host = " + host)
    print("port = {}".format(port))
    print("username = {}".format(username))
    print("password = {}".format(password))
    return kx.QConnection(host=host, port=port, username=username, password=password, tls=True)

In [9]:
conn = getPyKxConn(userName, clusterName)

host = vpce-02b4d0f70aab47141-idnyk9ae.vpce-svc-07988fe84c80b9cc6.eu-west-1.vpce.amazonaws.com
port = 443
username = finTorq_user
password = Host=vpce-02b4d0f70aab47141-idnyk9ae.vpce-svc-07988fe84c80b9cc6.eu-west-1.vpce.amazonaws.com&Port=5000&User=finTorq_user&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEO3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCWV1LXdlc3QtMSJIMEYCIQC1GR0sVKLXreF6roz5rUO4VvOJuE9vGqNN59uL44%2Fy4wIhAKiBYlajhcxtTGYZBkdAUwPFaffG%2FFiEKQL%2F6jwhArGLKvcCCHYQAhoMNzY2MDEyMjg2MDAzIgxux5nyECk7SZ%2Faddkq1AKS1yYctkxbDl1ZpK0dA4sLIOgtjomMhLuuwoOdnCqapVM%2FS%2B%2BbOn1e481CSR1o9oUGNxtBbUlstTuvMppRYHiiqdc7dxyHrlqV%2BuY4d2JJ3m%2FO2Dyoozob8AN3glDeApFRAmlW2Db8sy%2B871W9P%2BpMuX%2B5DPufDTUR%2Fe%2B8EyqqBtZ6D64P9mTTA5x2KK%2F6hsTFPVsKqq8JbxHEm2u84D2DA7AOqM67b7doCuEm1u9BXkXfKoX0xPMM34r%2FKk1O6tW6xaTUCUb%2BgMLgI9SQYEI5ezwuynNdmjokrvaxdO7QX%2FZJNyLJbPt9P2EB2nsiruKTiu1gYKmT3Z2F7sO02fxLdTJ4BS9%2FYNjH0vBKViCeTJHfqOfojvUyFzv0VMpSi9jc64Xqn7xjlF2rRmeCvhVrIJ8Xmvtl5Afmy7Z%2B3IIvlg

In [10]:
conn('.z.x').py()

[b'-noredirect', b'true', b'-procname', b'hdb1', b'-proctype', b'hdb']